In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

/Users/samarjeetsingh/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [4]:
tokens = tokenizer.encode("This is amazing, I loved it. Great", return_tensors='pt')
tokens[0]

tensor([  101, 10372, 10127, 39854,   117,   151, 46747, 10197,   119, 11838,
          102])

In [5]:
tokenizer.decode(tokens[0])

'[CLS] this is amazing, i loved it. great [SEP]'

In [6]:
result = model(tokens)

In [7]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.5011, -2.8063, -1.2865,  1.4575,  4.1100]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [8]:
int(torch.argmax(result.logits))+1

5

In [11]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [12]:
r

<Response [200]>

In [13]:
results[0]

<p class="comment__09f24__gu0rG css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).<br/><br/>Enjoyed the food.</span></p>

In [14]:
results[0].text

'The food is fresh and tasty. \xa0The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [15]:
reviews[0]

'The food is fresh and tasty. \xa0The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [16]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [17]:
df.head()

,review
0,The food is fresh and tasty. The scallop cevi...
1,Don't come here expecting legit Mexican food b...
2,Out of all the restaurants that I tried in Syd...
3,We came here on a Thursday night @ 5pm and by ...
4,I was pleasantly surprised at what a great job...


In [18]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [19]:
df['sentiment score'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [20]:
df.head(8)

,review,sentiment score
0,The food is fresh and tasty. The scallop cevi...,4
1,Don't come here expecting legit Mexican food b...,3
2,Out of all the restaurants that I tried in Syd...,5
3,We came here on a Thursday night @ 5pm and by ...,4
4,I was pleasantly surprised at what a great job...,5
5,Really nice (upmarket) Mexican restaurant. Goo...,4
6,Have been here twice and have absolutely loved...,5
7,If you're looking for a quiet little romantic ...,2


In [21]:
df.tail()

,review,sentiment score
5,Really nice (upmarket) Mexican restaurant. Goo...,4
6,Have been here twice and have absolutely loved...,5
7,If you're looking for a quiet little romantic ...,2
8,Ordered feed me for $59 along with that.. Food...,2
9,The service at this place was top notch - the ...,5
